In [1]:
%pip install --upgrade --quiet  flashrank


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
load_dotenv()
cf_embedding = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [4]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i + 1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

In [5]:
# documents = TextLoader(
#     "../../how_to/state_of_the_union.txt",
# ).load()

path = '/Users/pangmengting/Documents/workspace/python-learning/file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf'

documents = PyMuPDFLoader(path).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

retriever = FAISS.from_documents(texts, cf_embedding).as_retriever(search_kwargs={"k": 20})

query = "每日的行程总结一下"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

都河九曲十八弯的落日晚霞，天气晴好的傍晚，会看到一串的落日映在河面。日落后返回酒店休息。
车程：约280KM（国道，限速较多）
用餐：含早，正餐自理
住宿：巴音布鲁克镇网评3 钻商务酒店标间（旅游小镇，条件有限）
备注：1. 九曲十八弯日落极其震撼，但日落后非常寒冷（0-10 度），请带上厚外套、帽子、
手套等，湿地草原上蚊虫较多，要注意防蚊措施2. 景交终点至九曲十八弯观景台电瓶车（单程
10 元/人），也可选择徒步上山，费用自理
亮点/摄点：翻阅天山达坂、巴音布鲁克草原湿地、野生天鹅家园、开都河九曲十八弯日落
第5 天巴音布鲁克[大巴]特克斯[大巴]喀拉峻草原（草原毡房、星空之夜）
· 上午
早餐后离开，沿着218 国道，离开美丽的巴音布鲁克草原。星星点点的蒙古包，成群的黑头羊，炊烟牧
歌，远处连绵的雪山，风光无限。
· 下午
午后到达【特克斯】八卦城，进行简单的补给（明天午餐干粮）。
游侠客带领大家深入“天山世界自然遗产核心区”【喀拉峻大草原】。这里是天山世界自然遗产的核心
区，它定义了一种“草原在空中，森林在山下”的立体草原景观。位于天山山腰的喀拉峻大草原，鲜花遍
Metadata: {'source': '/Users/pangmengting/Documents/workspace/python-learning/file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf', 'file_path': '/Users/pangmengting/Documents/workspace/python-learning/file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf', 'page': 3, 'total_pages': 16, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'WPS 文字', 'producer': 'macOS 版本14.2.1（版号23C71） Quartz PDFContext', 'creationDate': "D:20240808094919+01'49'", 'modDate': "D:20240808

Doing reranking with FlashRank
使用 FlashRank 进行重新排名

现在，让我们使用 ContextualCompressionRetriever 包装我们的基本检索器，使用 FlashrankRerank 作为压缩器。

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

ms_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('MOONSHOT_API_BASE'),
    openai_api_key=os.getenv('MOONSHOT_API_KEY'),
    model_name="moonshot-v1-8k",
    temperature=0,
    streaming=True,
)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3.1-8b-instruct",
    temperature=0,
    streaming=True,
)

gq_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('GROQ_API_BASE'),
    openai_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama3-8b-8192",
    temperature=0,
    streaming=True,
)

zp_llm_openai = ChatOpenAI(
    openai_api_base='https://open.bigmodel.cn/api/paas/v4/',
    openai_api_key=os.getenv('ZHIPUAI_API_KEY'),
    model_name="GLM-4-0520",
    temperature=0,
    streaming=True,
)

In [7]:
from langchain_community.document_compressors import FlashrankRerank

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(query)
print([doc.metadata["id"] for doc in compressed_docs])

INFO:flashrank.Ranker:Downloading ms-marco-MultiBERT-L-12...
ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [00:43<00:00, 2.40MiB/s]


[1, 6, 29]


重新排名后，前 3 个文档与基础检索器检索到的前 3 个文档不同。

In [8]:
pretty_print_docs(compressed_docs)

Document 1:

0>
活动描述
第1 天全国各地[飞机]或[火车]乌鲁木齐（★全天集合日）
· 全天
今天是活动集合日。各位游侠自行前往新疆自治区首府【乌鲁木齐】的集合酒店，报“游侠客+名字”就
可以办理入住。
乌鲁木齐位于亚洲的中心，是举世闻名的歌舞之乡、瓜果之乡、黄金玉石之邦。如果时间比较宽裕，可以
自行安排在乌鲁木齐市区的活动。想要了解新疆人文历史的，首推【新疆博物馆】（免费，下午17:30
只出不进；周一闭馆）；想要感受异域风情的，可以去【二道桥国际大巴扎】，独特的西域文化，琳琅满
Metadata: {'source': '/Users/pangmengting/Documents/workspace/python-learning/file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf', 'file_path': '/Users/pangmengting/Documents/workspace/python-learning/file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf', 'page': 0, 'total_pages': 16, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'WPS 文字', 'producer': 'macOS 版本14.2.1（版号23C71） Quartz PDFContext', 'creationDate': "D:20240808094919+01'49'", 'modDate': "D:20240808094919+01'49'", 'trapped': '', 'id': 1, 'relevance_score': 0.9993359}
----------------------------------------------------------------------------------------------------
Document 2:

都河九曲十八弯的落日晚霞，天气晴好的傍晚，会看到一串的落日映在河面。日落后返回酒店休息。
车程：约280KM（国道，限速较多）
用餐：含早，正餐自

QA reranking with FlashRank
使用 FlashRank 重新排名 QA

In [9]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=zp_llm_openai, retriever=compression_retriever)

In [10]:
chain.invoke(query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


{'query': '每日的行程总结一下',
 'result': '根据提供的信息，以下是每日行程的总结：\n\n第1天：活动集合日\n- 全天：游侠们自行前往乌鲁木齐的集合酒店办理入住。\n- 可以自行安排在乌鲁木齐市区的活动，如参观新疆博物馆或二道桥国际大巴扎。\n\n第5天：巴音布鲁克至特克斯、喀拉峻草原\n- 上午：离开巴音布鲁克草原，沿途欣赏美丽的风光。\n- 下午：到达特克斯八卦城进行补给，随后深入喀拉峻大草原，欣赏立体草原景观。\n- 晚上：在草原毡房住宿，可观赏星空。\n\n行程中未提供第2至第4天和第6天及之后的详细行程。此外，行程还包括了一些关于花期的信息，表明团队会在不同时间欣赏到不同的花卉景观。'}